中文分词技术基本可以分为三个流派：  
1. 机械式分词法（基于词典）：机械分词的原理是将文档中的字符串与词典中的词条逐一匹配，若词典中找到某个字符串则匹配成功。基于词典的机械分词法实现简单、实用性抢，但缺点为词典的完备性不能得到保证。  
2. 基于语法和规则的分词法：基本思想为在分词的同时进行句法、语义分析，利用句法信息和语义信息来进行词性标注，解决分词歧义线性。因语法知识、句法规则十分笼统、复杂。目前效果仍较差。  
3. 基于统计的分词法：基本原理是根据字符串在语料库中出现的统计频率来决定是否构成词。词是字的组合，相邻的字同时出现的次数越多，就越有可能构成一个词。因此，字与字相邻共现的频率或概率可以很好的反映它们成为词的可信度。  

即便是母语为汉语的使用者，对于划分词汇的标准也是有分歧的。  
  
两种分词标准-粗粒度与细粒度：  
1. 粗粒度分词：将词作为语言处理最小的基本单位进行切分。
2. 细粒度分词：除了对词汇进行切分，也要对词汇内部的语素进行切分。  
  
粗粒度切分主要用于自然语言处理的各种应用；而细粒度分词最常用的领域是搜索引擎。  
常用的方案为：索引的时候使用细粒度的分词以保证召回，在查询的时候使用粗粒度分词以保证精度。  

## 机械分词系统
基于最大匹配方法作为最基本的分词算法，也称为MM(The Maximum Matching Method)方法。  
使用MM方法切分的精度并不高，很难达到实际应用的要求。随语料的增大，误差也逐渐变大。后续又提出来双向匹配法。  
  
### 双向匹配法
从最大匹配方法发展而来的，分为正向最佳匹配法和逆向最佳匹配法，两者仅是匹配方向相反的差异而已，两种方法所得的结果必然不同。  
**一个词汇的出现与其上下文环境中出现的词汇序列存在着紧密的关系，若算法不能反映和处理这种上下文关系，则最终不能达到满意的分词结果。**  
文本中第n个词的出现与其前后n-m到n+m个词有高度的相关性，而与这个范围之外的其他词相关性较低，故将[-m, m]范围也称为窗口范围。  


## 词汇的构成和命名实体
所谓词汇，一般都具有三个重要特性：稳固性、常用性和能产性。  
  
基于半监督的条件随机场(semi-CRF)算法，对于处理不同领域的专有名词识别具有较低的成本和较好的效果。

# ICTCLAS汉语分词系统
除N-最短路径算法之外，还实现了HMM的Viterbi算法、最大熵算法以及CRF算法等。  
ICTCLAS的算法思想来源于HMM。  
  
底下以HanLP的实现为例子进行讲解-  
HanLP的实现做了以下的修改：
1. 使用双数组Tire树读取和检索词典，提高性能。
2. 使用提供了更丰富的命名实体识别库：人名、译名、日本人名、地名、组织机构名等。  
3. 搜索引擎系统的支持等
4. 细分阶段使用了最短路径法。  
  
步骤
1. 系统读取待分词的字符串，输入的可以是一个句子也可以是一个篇章。若是篇章则系统会先进行切分，然后调用多线程，对每个切分的子句进行分词。  
2. 根据输入的配置信息，导入相应的词典。
3. 进入粗分阶段
    1. 首先对句子进行字符级切分，即将输入的句子切分为单个UTF-8编码的字符数组（函数toCharArray()），包括单个中文字符、单个英文字符、其他单个字符等。
    2. 一元切分：查核心词典，将字符切分的结果与词典词最大匹配，匹配的结果包含词形、词性、词频等信息形成一元词网，之后对一元词网进行原子切分，即按模式合并英文和数字的字符构成原子词。
    3. 二元切分：用一元分词的结果(二维数组)查询二元词典，与二元词典进行最大匹配，匹配的结果为一个Graph，形成一个词图。
    4. NShort算法计算：将查出的每个结果按平滑算法计算二元分词的词频数得到词图的每个节点的权值（概率的导数），应用NShort算法累加词图中的每个节点构成的所有路径，权值最小（概率最大）的那条路径对应的词图节点就是粗分的结果。
    5. 对粗分结果执行后处理应用规则，识别时间类专有名词。  
4. 进入未登入词识别阶段，使用隐马尔可夫链语言模型。
    1. 根据人名识别词典，将粗分的结果与之匹配，Viterbi算法识别外国的人名。
    2. 根据地名识别词典，将粗分的结果与之匹配，Viterbi算法识别地名。
    3. 根据组织机构名词典，将粗分的结果与之匹配，Dijkstra算法识别组织机构名。  
5. 将命名实体识别后的分词加入词图中，对词图再次进行分词（Dijkstra最短路径法）。该阶段为细分阶段。
6. 使用词性标注模型、Viterbi算法对分词结果进行词性标注。
6. 转换路径为分词结果，并输出分词结果。  

## 词典的数据结构
### 散列表
中等规模数据的存储和查询常用的数据结构是HashTable，也称作散列表。  
散列表的基本原理是在表项的存储位置与其索引键（查询索引）之间建立一个对应函数关系，这个函数称为哈希函数。  
通过哈希函数计算得到每个索引键与表项的存储位置相对应的关系。  
当输入查询键时，哈希表可以近似的得到随机访问的查询效率，这种数据结构因为对键的约束不高（不要求一定是整数类型），故常用于中等规模存储和查询。  
### Trie树
随数据量的增加，对于大规模的数据存储（百万单位以上），哈希表的不足逐渐显露。大规模数据对内存空间占用高，Hash表因为需要哈希函数计算地址进行映射，必然会导致一些数据空间的损失（空桶），中小规模数据的情况不明显，但随数据规模的上升，这种空间浪费会成倍的增加，导致有效存储空间的比例显著下降。  
Trie树为一种搜索树，本质是一个确定有限状态自动机（DFA），每个节点代表自动机的一个状态，根据变量不同进行状态转移，当到达结束状态或无法转移时，完成一次查询操作。  
Trie树搜索一个检索键的时间与键自身长度有关，最快是O(1)，即在第一层即可判断是否搜索到，最坏是O(n)，n为Trie树的层数。  
  
### 双数组Trie树(Double-Array Trie)
将Trie树用两个整型数组来表示，目的是为了设计一种Trie结构的压缩形式。  
该结构有效结合了数字搜索树（Digital Search Tree）检索时间高效的特点和链式表示的Trie空间结构紧凑的特点，其本质是一个确定有限状态机自动机（DFA）。DFA的每个节点代表自动机的一个状态，根据变量的不同进行状态转移，当到达结束状态或无法转移时，完成一次查询操作。  
在双数组的所有键中包含的字符之间的所有联系都是通过简单的数学加法运算来表示，不仅提高了检索速度，而且省去了链式结构中使用的大量指针，节省了存储空间。  

# NShort最短路径算法
步骤：
1. 从词图中构建出节点和权重排序数组
2. 从词图中过滤掉排序数组中权重较高的路径，并根据权重较低的路径生成最终的分词结果。